In [ ]:
import os
import glob
import shutil
from datetime import datetime, timedelta

In [ ]:
today = str((datetime.now() - timedelta(days=0)).date()).replace('-', '_')
# today = str((datetime.now() - timedelta(days=7)).date()).replace('-', '_')
today

In [ ]:
# # remove weights execept last one
# model_dirs = glob.glob("/root/data/models/keypoints_detection/*")
# for md in model_dirs:
#     full_date = '_'.join(os.path.basename(md).split('_')[:3])
#     year, month, day = full_date.split('_') 
#     if int(month) == datetime.now().date().month:
#         if int(day) > (datetime.now().date().day - 4):
#             print(md, 'training')
#             continue
            
#     if not os.path.isdir(md):
#         continue
        
#     weight_path = os.path.join(md, "weights")
#     all_meta = glob.glob(weight_path + '/*.meta')
#     all_weights = glob.glob(weight_path + '/*')
#     max_epoch = len(all_weights) - 2
#     for i in range(max_epoch):
#         all_files = glob.glob(weight_path + '/fish_test-{}.*'.format(i))
#         for f in all_files:
#             os.remove(f)
        
# #     if len(os.listdir(weight_path)) < 1:
# #         shutil.rmtree(md)
# #         print("delete {}".format(md))

Clean empty dirs

In [ ]:
import os
import glob
import shutil

In [ ]:
# len(glob.glob('/root//data/fish_detections/tmp/environment=production/site-id=23/pen-id=1/*/*/*/*'))

In [ ]:
# model_dirs = glob.glob("/root/data/models/keypoints_detection/*")
# for md in model_dirs:
#     if not os.path.isdir(md):
#         continue
#     weight_path = os.path.join(md, "weights")
#     if len(os.listdir(weight_path)) < 1:
#         shutil.rmtree(md)
#         print("delete {}".format(md))

Plot

In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np

- 2019_05_22_20_05_37: GTSF best
- 2019_05_13_22_13_59: BATI, no augmentation
- 2019_05_09_20_46_29: GTSF + BATI, no augmentation
- 2019_05_10_00_36_40: GTSF, augmentation, 512x512, buffer range -> best GTSF model
- 2019_05_09_18_01_03: GTSF, no augmentation, 1024x1024, 150 epochs, 4 stages, faster decay 
- 2019_05_09_15_27_07: GTSF, no augmentation, 512x512, 150 epochs, 3 stages
- 2019_05_08_16_24_20: GTSF, no augmentation, 1024x1024, 150 epochs, 4 stages
- 2019_05_07_17_04_36: GTSF, no augmentation, 1024x1024, 150 epochs
- 2019_05_03_16_59_35: BATI, no augmentation 1024x1024, 150 epochs
- 2019_05_02_21_25_53: BATI, 512x512, 150 epochs
- 2019_05_01_23_03_06: BATI, 256x256, 300 epochs
- 2019_04_05_18_32_22: rotation, 512x512, 3 stages, 300 epochs, gaussian var=4.0
- 2019_04_05_18_28_50: all augmentation, 512x512, 3 stages, 300 epochs
- 2019_04_04_18_26_16: rotation, 512x512, 150 epochs, 3 stages
- 2019_04_04_17_30_18: no augmentation, 512x512, 150 epochs, 4 stages
- 2019_04_03_18_45_50: no augmentation, 512x512, 150 epochs -> best model so far
- 2019_04_03_20_20_11: no augmentation, 256x256, 150 epochs
- 2019_04_02_22_17_08: rotation + image augmentation (contrast brightness)
- 2019_04_03_00_24_44: rotation only
- 2019_03_27_22_42_47: no augmentation, 50 epochs
- 2019_03_27_22_28_27: no augmentation, 50 epochs, low rez

In [ ]:
with open("/root/data/models/keypoints_detection/2019_05_27_06_19_12//config.json", "r") as f:
    config = json.load(f)

In [ ]:
print(config)

In [ ]:
with open("/root/data/models/keypoints_detection/2019_05_27_06_19_12/history.json", "r") as f:
    history = json.load(f)

In [ ]:
train_total_loss = history["train_total_loss"]
train_stages_loss = np.array(history["train_stages_loss"])
learning_rate = history.get('learning_rate', None)
val_total_loss = history["val_total_loss"]
epochs = len(val_total_loss)

In [ ]:
steps_per_epoch = int(len(train_total_loss) / len(val_total_loss))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(train_total_loss)
plt.plot(range(steps_per_epoch, (epochs+1)*steps_per_epoch, steps_per_epoch), val_total_loss, linewidth=4)
plt.ylim([0, 500])
plt.xlim([0, epochs*steps_per_epoch])
plt.xticks(ticks=range(0, epochs*steps_per_epoch, steps_per_epoch*10),
           labels=range(0, epochs, 10))
min_val = np.min(val_total_loss)
plt.plot([0, epochs*steps_per_epoch], [min_val, min_val], "g")
plt.text(20, min_val+1, "Min validation loss: {:.2f}. Epoch: {}".format(min_val, np.argmin(val_total_loss)),
         {"color": "k", "fontsize": 14})
plt.legend(["train", "val"])
# plt.yscale('log')
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(learning_rate)
plt.xlim([0, epochs*steps_per_epoch])
plt.xticks(ticks=range(0, epochs*steps_per_epoch, steps_per_epoch*5),
           labels=range(0, epochs, 5))
plt.ylabel('Learning rate')
plt.xlabel('Epochs')
plt.show()

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(train_total_loss)
plt.plot(range(steps_per_epoch, (epochs+1)*steps_per_epoch, steps_per_epoch), val_total_loss, linewidth=4)
plt.ylim([0, 150])
plt.xlim([0, 30*steps_per_epoch])
plt.xticks(ticks=range(0, 30*steps_per_epoch, steps_per_epoch*5),
           labels=range(0, epochs, 5))
plt.show()